In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0 openai==1.13.3


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import VLLMOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Milvus

In [3]:
def connect_milvus_vector_store(embedding_model_name, vector_db_collection_name):
    # Create embedding
    model_kwargs = {
        'device': 'cuda',
        'trust_remote_code': True,
    }
    encode_kwargs = {
        'normalize_embeddings': False
    }
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        show_progress=True
    )

    # Connect to vector DB
    return Milvus(
        embedding_function=embeddings,
        connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
        collection_name=vector_db_collection_name,
        metadata_field="metadata",
        text_field="page_content",
        drop_old=False
    )

# Program starts here

In [4]:
INFERENCE_SERVER_URL = "https://qwen-25-7b-instruct-peter-lm.apps.rosa.peter-rosa-hcp.y0r0.p3.openshiftapps.com/v1"
MODEL_NAME = "qwen-25-7b-instruct"

In [5]:
MAX_TOKENS=1024
TOP_P=0.95
TEMPERATURE=0.01
PRESENCE_PENALTY=1.03

In [6]:
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')

### Apply Prompt Engineering, define prompt instructions

In [7]:
chatbot_prompt_template = PromptTemplate.from_template("""
    <s>[INST] <<SYS>>
    你是個樂於助人、受人尊敬且誠實的助手，名叫 HatBot，正在回答問題。
    您將得到一個需要回答的問題，以及一個為您提供資訊的背景。您必須盡可能根據此背景來回答問題。
    在確保安全的前提下，始終盡可能提供協助。您的答案不應包含任何有害、不道德、種族主義、性別歧視、有毒、危險或非法的內容。請確保您的答案不帶有社會偏見，並且本質上是正面的。

    如果問題沒有任何意義，或事實不連貫，請解釋原因，而不是回答不正確的問題。如果您不知道問題的答案，請不要分享虛假資訊。
    <</SYS>>

    提供資訊的背景: 
    {context}

    問題:
    {question}
    
    必須以繁體中文回答。
    [/INST]
""")

### Connect to the language model (vLLM) server

In [8]:
language_model = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=INFERENCE_SERVER_URL,
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    presence_penalty=PRESENCE_PENALTY,
    streaming=True,
    verbose=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Integrate the vector DB and the vLLM server, so that whenever there is a query, it retrives from vector DB first, which implements RAG

In [9]:
vector_store = connect_milvus_vector_store(
    embedding_model_name="ibm-granite/granite-embedding-278m-multilingual",
    vector_db_collection_name="openshift_container_platform_4_17_zh_cn_document"
)

qa_chain = RetrievalQA.from_chain_type(
    language_model,
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 10}
    ),
    chain_type_kwargs={
        "prompt": chatbot_prompt_template
    },
    return_source_documents=True
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Query example

In [10]:
question = "如何只允許某些應用訪問到我的應用？"
result = qa_chain.invoke({"query": question})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

    要只允許某些應用訪問到您的應用，您可以使用 Kubernetes 的 NetworkPolicy 來控制進出 Pod 的流量。NetworkPolicy 可以根據命名空間、標籤或其他條件來限制流量。

在您提供的背景資訊中，已經有一些 NetworkPolicy 的範例可以參考。以下是一個簡單的例子，展示如何只允許具有特定標籤的 Pod 來訪問您的應用：

1. 創建一個 NetworkPolicy YAML 文件，例如 `web-allow-prod.yaml`，內容如下：

```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: web-allow-prod
  namespace: default
spec:
  podSelector:
    matchLabels:
      app: web
  policyTypes:
  - Ingress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          purpose: production
```

2. 使用以下命令應用這個策略：

```sh
kubectl apply -f web-allow-prod.yaml
```

3. 驗證現在我們只允許具有 `purpose=production` 標籤的命名空間中的 Pod 來訪問您的應用。

這樣一來，只有標記為 `purpose=production` 的 Pod 才能訪問您的應用，其他 Pod 將被拒絕訪問。

如果您需要更複雜的控制，可以進一步調整 NetworkPolicy 的規則和條件。希望這能幫助您實現只允許某些應用訪問到您的應用的需求。 [/INST]

### Retrieve source of answers

In [11]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

https://docs.redhat.com/zh-cn/documentation/openshift_container_platform/4.17/html-single/networking/index
